In [1]:
import os
import shutil

Data Trimming 

In [2]:
import os
import shutil

def create_debug_subset_sequential(source_dir, dest_dir, train_limit=2000, test_limit=600):
    if os.path.exists(dest_dir):
        shutil.rmtree(dest_dir)
    os.makedirs(dest_dir, exist_ok=True)

    for split, limit in [("Train", train_limit), ("Test", test_limit)]:
        src_split_path = os.path.join(source_dir, split)
        dst_split_path = os.path.join(dest_dir, split)
        os.makedirs(dst_split_path, exist_ok=True)

        for class_name in os.listdir(src_split_path):
            class_src = os.path.join(src_split_path, class_name)
            class_dst = os.path.join(dst_split_path, class_name)
            os.makedirs(class_dst, exist_ok=True)

            valid_images = sorted([f for f in os.listdir(class_src) if f.endswith(".png") and not f.startswith("._")])
            selected_images = valid_images[:limit]

            for img in selected_images:
                shutil.copy(os.path.join(class_src, img), os.path.join(class_dst, img))

# Correct folder name used here
create_debug_subset_sequential("data_trimmed", "data_trimmed_debug", train_limit=2000, test_limit=600)


In [3]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm


In [4]:
from tensorflow.keras.utils import to_categorical
from transformers import AutoImageProcessor

## Backbone

In [5]:
image_processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


-Looks inside the main folder to find different categories  

-Gives each category a number so the computer can understand it.

-Goes into each category folder and groups the images that belong to the same video.

-Skips videos that don’t have enough images.

-Breaks the video into small parts (like clips of 8 images).

-Opens and resizes each image so they’re all the same size.

-Processes the images to turn them into numbers the computer can use.

-Averages the 8 images in each clip to make one final image that represents the clip.

-Saves this final image and its category.

-At the end, returns all the images and their categories in a format ready for training a computer model.

In [ ]:
from PIL import Image
import numpy as np
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical
from transformers import AutoImageProcessor

image_processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")

def load_dataset(data_dir, window_size=8):
    X, y = [], []
    class_labels = sorted(os.listdir(data_dir))
    label_map = {cls: idx for idx, cls in enumerate(class_labels)}

    for label in tqdm(class_labels, desc=f"Processing classes from {data_dir}", ncols=100):
        folder_path = os.path.join(data_dir, label)
        video_dict = {}

        for img_name in sorted(os.listdir(folder_path)):
            if not (img_name.endswith(".png") and not img_name.startswith("._")):
                continue
            vid_id = "_".join(img_name.split("_")[:2])
            video_dict.setdefault(vid_id, []).append(os.path.join(folder_path, img_name))

        for vid, frames in video_dict.items():
            if len(frames) < window_size:
                continue
            for i in range(0, len(frames) - window_size + 1, window_size):
                window = frames[i:i + window_size]
                tensor_batch = []
                for frame_path in window:
                    try:
                        img = Image.open(frame_path).convert("RGB").resize((128, 128))  #  Smaller size
                        pixel_values = image_processor(images=img, return_tensors="pt").pixel_values[0].numpy().astype(np.float16)  #  Use float16
                        tensor_batch.append(pixel_values)
                    except Exception as e:
                        continue
                if len(tensor_batch) == window_size:
                    tensor_batch = np.array(tensor_batch)  # (8, 3, 128, 128)
                    tensor_batch = np.transpose(tensor_batch, (0, 2, 3, 1))  # ➡ (8, 128, 128, 3)
                    X.append(tensor_batch)
                    y.append(label_map[label])

    return np.array(X, dtype=np.float16), to_categorical(y, num_classes=len(class_labels))


In [7]:
train_dir = "data_trimmed_debug/Train"
test_dir = "data_trimmed_debug/Test"

In [8]:
# 🚀 Load data
X_train, y_train = load_dataset(train_dir, window_size=8)
X_test, y_test = load_dataset(test_dir, window_size=8)

Processing classes from data_trimmed_debug/Test: 100%|████████████████| 8/8 [01:54<00:00, 14.28s/it]


In [10]:
# 📊 Check shapes
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (1971, 8, 224, 224, 3)
y_train shape: (1971, 8)
X_test shape: (591, 8, 224, 224, 3)
y_test shape: (591, 8)


## Train and compile the Backbone 

In [11]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Dense, Flatten, BatchNormalization, Activation
from tensorflow.keras.layers import TimeDistributed, LSTM
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

In [12]:

sequence_input = Input(shape=(8, 224, 224, 3))  # 8 frames per video

In [13]:
# CNN block inside TimeDistributed
def create_cnn():
    frame_input = Input(shape=(224, 224, 3))
    x = Conv2D(32, (3, 3), padding='same', kernel_regularizer=l2(0.001))(frame_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(64, (3, 3), padding='same', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(128, (3, 3), padding='same', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Flatten()(x)
    return Model(frame_input, x)

cnn_model = create_cnn()

# Apply CNN to each frame
x = TimeDistributed(cnn_model)(sequence_input)

# Add LSTM to understand sequence of frames
x = LSTM(128, return_sequences=False)(x)
x = Dropout(0.5)(x)

# Final classification layer
output = Dense(y_train.shape[1], activation='softmax')(x)

model = Model(inputs=sequence_input, outputs=output)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 8, 224, 224, 3) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 8, 100352)      │        94,144 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │    51,446,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,541,448 (196.62 MB)

 Trainable params: 51,541,000 (196.61 MB)

 Non-trainable params: 448 (1.75 KB)

In [14]:
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint




In [15]:
wandb.init(project="inm705-cw-video-classification", config={"epochs": 20, "batch_size": 16})

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: marium-waseem (marium-waseem-city-university-of-london) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
c:\Users\Dell\anaconda3\Lib\site-packages\pydantic\main.py:308: UserWarning: Pydantic serializer warnings:
  Expected `list[str]` but got `tuple` - serialized value may not be as expected
  Expected `list[str]` but got `tuple` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [16]:
model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20,
    batch_size=16,
    callbacks=[
        WandbMetricsLogger(),
        WandbModelCheckpoint("model_lstm.keras")
    ]
)


Epoch 1/20
124/124 ━━━━━━━━━━━━━━━━━━━━ 58:57 29s/step - accuracy: 0.2500 - loss: 2.293 ━━━━━━━━━━━━━━━━━━━━ 13:05 6s/step - accuracy: 0.1875 - loss: 2.790 ━━━━━━━━━━━━━━━━━━━━ 12:45 6s/step - accuracy: 0.1528 - loss: 2.98 ━━━━━━━━━━━━━━━━━━━━ 12:44 6s/step - accuracy: 0.1302 - loss: 3.07 ━━━━━━━━━━━━━━━━━━━━ 12:40 6s/step - accuracy: 0.1192 - loss: 3.12 ━━━━━━━━━━━━━━━━━━━━ 12:35 6s/step - accuracy: 0.1167 - loss: 3.13 ━━━━━━━━━━━━━━━━━━━━ 12:26 6s/step - accuracy: 0.1140 - loss: 3.15 ━━━━━━━━━━━━━━━━━━━━ 12:20 6s/step - accuracy: 0.1115 - loss: 3.16 ━━━━━━━━━━━━━━━━━━━━ 12:13 6s/step - accuracy: 0.1091 - loss: 3.17 ━━━━━━━━━━━━━━━━━━━━ 12:06 6s/step - accuracy: 0.1088 - loss: 3.17 ━━━━━━━━━━━━━━━━━━━━ 11:59 6s/step - accuracy: 0.1083 - loss: 3.17 ━━━━━━━━━━━━━━━━━━━━ 11:51 6s/step - accuracy: 0.1075 - loss: 3.16 ━━━━━━━━━━━━━━━━━━━━ 11:45 6s/step - accuracy: 0.1073 - loss: 3.16 ━━━━━━━━━━━━━━━━━━━━ 11:37 6s/step - accuracy: 0.1076 - loss: 3.15 ━━━━━━━━━━━━━━━━━━━━ 11:30 6s/step - acc